In [1]:
from IPython.display import HTML

embed='<iframe style="border: 0; width: 400px; height: 339px;" src="https://bandcamp.com/EmbeddedPlayer/album=2664818884/size=large/bgcol=333333/linkcol=ffffff/artwork=small/track=3028022632/transparent=true/" seamless><a href="http://nekkralai.bandcamp.com/album/kliud-iau">Kliudziau by NekKralai</a></iframe>'

HTML(embed)

In [2]:
import re

## path to XML
xml_path='../XMLs/GireWA_hkySCCS_20M_1.xml'

listen=False

## dict to map sequence names to sequences
strains={}

strain_order=[]

## iterate over every line in the XML
for line in open(xml_path,'r'):
    
    ## start listening for sequence name and sequence whenever <sequence> tag was encountered
    if '<sequence>' in line:
        name=''
        listen=True
        
    ## stop listening for sequence name and sequence whenever </sequence> tag was encountered
    if '</sequence>' in line:
        name=''
        listen=False
    
    ## search for sequence name
    name_cerberus = re.search('^\t\t\t<taxon idref="([A-Za-z0-9\_\.]+)"/>$',line)
    if name_cerberus is not None and listen==True:
        name = name_cerberus.group(1)
        strain_order.append(name)
        strains[name] = ''
    
    ## search for sequence
    seq_cerberus = re.search('^\t\t\t([A-Z\-]+)$',line)
    if seq_cerberus is not None and listen==True:
        strains[name]+=seq_cerberus.group(1)

In [3]:
import datetime

## only isolate C07 gives dates to the rest of Baize et al isolates that match public records
C07_date = '2014-03-20'

## iterate over all strains, find C15 sequence from Baize et al
for strain in strains.keys():
    ## find C07 isolate
    if 'C07' in strain:
        print 'Calibrating on:',strain,
        diff=int(strain.split('_')[-1])
        print diff
        ## identify the XML calibration date based on known isolation day of C07 isolate
        date_1 = datetime.datetime.strptime(C07_date,"%Y-%m-%d")

## date when tips in the XML were calibrated
calibration_date = date_1 + datetime.timedelta(days=diff)

print 'Calibration date: %s\n\n'%(datetime.datetime.strftime(calibration_date,"%Y-%m-%d"))

def realDate(diff):
    """Adds a number of days to the calibrated time point, returns calendar date."""
    return datetime.datetime.strftime(calibration_date - datetime.timedelta(days=diff),"%Y-%m-%d")

## specify output file
output_fasta=open('EBOV_rates_project_2016.fasta','w')

## report fasta
for strain in strain_order:
    diff=int(strain.split('_')[-1])
    #print '>%s|%s\n%s'%(strain,realDate(diff),strains[strain])
    print>>output_fasta,'>%s|%s\n%s'%(strain,realDate(diff),strains[strain])
    
output_fasta.close()

Calibrating on: KJ660347p2_Gueckedou_C07_276 276
Calibration date: 2014-12-21




In [4]:
import re

xml_fasta='../alignments/EBOV_rates_project_full.fasta'
correct_fasta='../alignments/Gire_Mali_old.fasta'

xml_full_name={}
correct_full_name={}

xml_alignment={}
correct_alignment={}

xml_dates={}
correct_dates={}

xml_order=[]

## import XML alignment
for line in open(xml_fasta,'r'):
    l=line.strip('\n')
    if '>' in l:
        cerberus=re.search('[A-Z0-9]+',l)
        if cerberus is not None:
            name=cerberus.group()
        xml_full_name[name]=l[1:]
        xml_order.append(name)
        xml_dates[name]=l.split('|')[-1]

    elif name!='':
        seq=l
        xml_alignment[name]=seq
        name=''

## import correct alignment
for line in open(correct_fasta,'r'):
    l=line.strip('\n')
    if '>' in l:
        name=l[1:].split('|')[0]
        correct_full_name[name]=l[1:]
        correct_dates[name]=l.split('|')[-1]

    elif name!='':
        seq=l
        correct_alignment[name]=seq
        name=''
        
## open output file
mapping_out=open('strain_map.csv','w')

## add header
print>>mapping_out,'correct accession,correct AKA,matched xml accession,matched xml AKA'

## iterate over each sequence
for i in xml_order:
    seqA=correct_alignment[i]
    
    best_matches_0=[]
    best_matches_1=[]
    
    ## compare against every other sequence
    for j in xml_order:
        seqB=xml_alignment[j]

        ## get sequence mismatches
        mismatches=[0 if seqA[k]==seqB[k] else 1 for k in range(len(seqB))]

        if sum(mismatches)==0:
            best_matches_0.append(j)
        elif sum(mismatches)==1:
            best_matches_1.append(j)
    
    print i,best_matches_0
    
    ## output to file
    print>>mapping_out,'%s,%s,%s,%s'%(i,correct_full_name[i],';'.join(best_matches_0),';'.join([xml_full_name[w] for w in best_matches_0]))
    
mapping_out.close()

KP260799 ['KP260799']
KP260800 ['KP260800']
KP260801 ['KP260801']
KP260802 ['KP260802']
KM034555 ['KM034555']
KM034556 ['KM034562']
KM034557 ['KM233052']
KM034558 ['KM233108']
KM034559 ['KM034556']
KM034560 ['KM233054']
KM034561 ['KM034550']
KM034562 ['KM034549']
KM034563 ['KM034551']
KM233035 ['KM233043']
KM233036 ['KM233083']
KM233037 ['KM233055', 'KM233068', 'KM233069']
KM233038 ['KM233055', 'KM233068', 'KM233069']
KM233039 ['KM233070']
KM233040 ['KM034559']
KM233041 ['KM233086']
KM233042 ['KM233046']
KM233043 ['KM233104']
KM233044 ['KM233057']
KM233045 ['KM233084']
KM233046 ['KM233100']
KM233047 ['KM233035']
KM233048 ['KM233098']
KM233049 ['KM034560']
KM233050 ['KM233049']
KM233051 ['KM233050']
KM233052 ['KM233051']
KM233053 ['KM233061']
KM233054 ['KM233072']
KM233055 ['KM233092']
KM233056 ['KM233062']
KM233057 ['KM233065', 'KM233066']
KM233058 ['KM233109']
KM233059 ['KM233110']
KM233060 ['KM233111']
KM233061 ['KM233089']
KM233062 ['KM233038']
KM233063 ['KM233065', 'KM233066']
KM23

In [5]:
## iterate over strains
for i in correct_alignment.keys():
    ## check for mismatched dates
    if correct_dates[i]!=xml_dates[i]:
        print 'mismatched dates:',i,xml_full_name[i],correct_full_name[i],correct_dates[i],xml_dates[i]

mismatched dates: KJ660346 KJ660346p2_Kissidougou_C15_277|2014-03-19 KJ660346|Makona-Kissidougou-C15|2014-03-17 2014-03-17 2014-03-19
mismatched dates: KJ660348 KJ660348p2_Gueckedou_C05_279|2014-03-17 KJ660348|Makona-Gueckedou-C05|2014-03-19 2014-03-19 2014-03-17


In [12]:
import re

haplotypes=[]

xml_fasta='../alignments/EBOV_rates_project_full.fasta'
correct_fasta='../alignments/Gire_Mali_old.fasta'

xml_full_name={}
correct_full_name={}

xml_alignment={}
correct_alignment={}

xml_dates={}
correct_dates={}

xml_order=[]
correct_order=[]


## Construct haplotype map
for line in open(correct_fasta,'r'):
    l=line.strip('\n')
    if '>' in l:
        cerberus=re.search('[A-Z0-9]+',l)
        if cerberus is not None:
            name=cerberus.group()
        correct_order.append(name)


    elif name!='':
        ## ignore last nucleotide, because XML sequences do not have it
        seq=l[:-1]
        name=''
        
        if seq not in haplotypes:
            haplotypes.append(seq)
        


## import XML alignment
for line in open(xml_fasta,'r'):
    l=line.strip('\n')
    if '>' in l:
        cerberus=re.search('[A-Z0-9]+',l)
        if cerberus is not None:
            name=cerberus.group()
        xml_full_name[name]=l[1:]
        xml_dates[name]=l.split('|')[-1]
        xml_order.append(name)
        
    elif name!='':
        seq=l
        xml_alignment[name]=seq
        name=''
        

## import correct alignment
for line in open(correct_fasta,'r'):
    l=line.strip('\n')
    if '>' in l:
        name=l[1:].split('|')[0]
        correct_full_name[name]=l[1:]
        correct_dates[name]=l.split('|')[-1]

    elif name!='':
        ## ignore last nucleotide, because XML sequences do not have it
        seq=l[:-1]
        correct_alignment[name]=seq
        name=''
        
        

## sort haplotypes (just in case the XML sequences were sorted that way)
#haplotypes=sorted(haplotypes)

## output assigned ID to each haplotype
hap_IDs=open('haplotype_IDs.csv','w')
print>>hap_IDs,'ID,sequence'
for i,hap in enumerate(haplotypes):
    print>>hap_IDs,'%d,%s'%(i+1,hap)

hap_IDs.close()

## output the haplotype ID of each sequence in the order in which strains appear in the XML
hap_map=open('haplotype_map_xml.csv','w')
print>>hap_map,'name,collection date,hap ID'
for strain in xml_order:
    name = xml_full_name[strain]
#    print name,strain

    hapID=haplotypes.index(xml_alignment[strain])+1
    
    print>>hap_map,'%s,%s,%s,%s'%(name,strain,xml_dates[strain],hapID)
             
hap_map.close()

## output the haplotype ID of each sequence in the order in which strains appear in the correct alignment
hap_map=open('haplotype_map_correct.csv','w')
print>>hap_map,'name,collection date,hap ID'
for strain in correct_order:

    name = correct_full_name[strain]
#    print name,strain

    hapID=haplotypes.index(correct_alignment[strain])+1
    
    print>>hap_map,'%s,%s,%s,%s'%(name,strain,correct_dates[strain],hapID)
             
hap_map.close()